<h1>Google Ranking Algorithm</h1>

In [1]:
import numpy as np
import urllib.request
import os
import time
from tqdm.notebook import tqdm

In [2]:
#dictionary of urls
url_targets = {}
url_pointers = {}
#damping factor
d = 0.85
#creeper iterations
creeper_iterations=2
#base url
base_url = "https://en.wikipedia.org"
#initial url
initial_url = "/wiki/Black_Hole_Initiative"

<h1>Crawler</h1>

In [3]:
def crawler(initial_url, creeper_iterations):
    htmls = os.listdir('wiki')
    #list of websites a website ponts to 
    url_targets = {}
    open_urls = [initial_url]
    for n in range(creeper_iterations):
        print(n)
        new_urls = set()
        for url in tqdm(open_urls):
            url_targets[url] = set()
            try:
                file_name = '_'.join(url.split('/')[2:])+'.html'
                if file_name.replace('*','<star>') in htmls:
                    file = open('wiki/'+file_name,encoding="utf-8").read()
                else:    
                    file = urllib.request.urlopen(base_url+url).read().decode("utf-8")
                    html = open('wiki/'+file_name.replace('*',''),"w+",encoding="utf-8")
                    html.write(file)
                    html.close()
                file = file.replace(" ","")
                url_list =file.split('href="')
                for k in range(len(url_list)):
                    url_list[k] = url_list[k].split('"')[0]
                for found_url in url_list:
                    if not found_url in url_targets.keys() and "/wiki/" == found_url[:6] \
                    and not "#" in found_url and not ":" in found_url:
                        new_urls.add(found_url)
                    if not found_url in url_targets[url]:
                        url_targets[url].add(found_url)
            except Exception as e: 
                print(str(e))
            open_urls = new_urls
    for key in url_targets:
        new_targets = set()
        for target in url_targets[key]:
            if target in url_targets.keys():
                new_targets.add(target)
        url_targets[key] = new_targets
    return url_targets


In [4]:
url_targets = crawler(initial_url, creeper_iterations)

0



1


<h1>Relaxation Method</h1>

We assume page $A$ has pages $T_1 ... T_n$ which point to it (i.e., are citations). The parameter $d$ is a damping factor which can be set between 0 and 1. We usually set $d$ to 0.85. There are more details about $d$ in the next section. Also $C(A)$ is defined as the number of links going out of page $A$. The PageRank of a page $A$ is given as follows:<br><br>
$$PR(A) = (1-d) + d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n))$$<br>
Note that the PageRanks form a probability distribution over web pages, so the sum of all web pages' PageRanks will be one.

In [5]:
start_time = time.time()

page_rank = {}

for key in url_targets:
    page_rank[key] = 1/len(url_targets[key])

for n in range(5):
    new_rank = {}
    for key in tqdm(page_rank):
        new_rank[key] = 1-d
        for sum_key in page_rank:
            if key in url_targets[sum_key]:
                new_rank[key] += d*page_rank[sum_key]/len(url_targets[sum_key])
    page_rank = new_rank.copy()

page_rank = {url: rank for url, rank in sorted(page_rank.items(), key=lambda item: item[1], reverse=True)}

print('time to run: ',time.time()-start_time)


time to run:  0.1984424591064453


In [6]:
list(page_rank)[:10]

['/wiki/Main_Page',
 '/wiki/Black_hole',
 '/wiki/Dark_matter',
 '/wiki/Universe',
 '/wiki/Supernova',
 '/wiki/General_relativity',
 '/wiki/Galaxy',
 '/wiki/Dark_energy',
 '/wiki/White_dwarf',
 '/wiki/Neutron_star']

In [7]:
[(key,page_rank[key]) for key in page_rank.keys()][:10]

[('/wiki/Main_Page', 8.828104827527515),
 ('/wiki/Black_hole', 1.15439880271944),
 ('/wiki/Dark_matter', 1.138515241404342),
 ('/wiki/Universe', 1.0739052881659963),
 ('/wiki/Supernova', 1.06405236425116),
 ('/wiki/General_relativity', 1.0621183265611853),
 ('/wiki/Galaxy', 0.9913803371022254),
 ('/wiki/Dark_energy', 0.9449846788778061),
 ('/wiki/White_dwarf', 0.933710808031581),
 ('/wiki/Neutron_star', 0.9286638536722459)]

<h1>Linear Algebra</h1>

$$PR(A) = (1-d) + d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n))$$<br>
$$PR(A) - d (PR(T_1)/C(T_1) + ... + PR(T_n)/C(T_n)) = 1-d$$<br>
$$\begin{bmatrix}1 & -d/C_1(T_2) & ... \\ 
-d/C_2(T_1) & 1 & ... \\ \vdots & \vdots & \ddots \end{bmatrix} \begin{bmatrix}PR(A_1) \\ PR(A_2) \\ \vdots \end{bmatrix} = 
\begin{bmatrix}1-d \\ 1-d \\ \vdots \end{bmatrix}$$<br>
$$\begin{bmatrix}PR(A_1) \\ PR(A_2) \\ \vdots \end{bmatrix} = 
\begin{bmatrix}1-d \\ 1-d \\ \vdots \end{bmatrix}\begin{bmatrix}1 & -d/C_1(T_2) & ... \\ 
-d/C_2(T_1) & 1 & ... \\ \vdots & \vdots & \ddots \end{bmatrix}^{-1}$$

In [8]:
start_time = time.time()

keys = list(page_rank.keys())
b = np.zeros(len(keys))+1-d
A = np.identity(len(keys))
for row in range(len(keys)):
    for column in range(len(keys)):
        if row != column and keys[column] in url_targets[keys[row]]:
            A[row,column] = -d/len(url_targets[keys[row]])
PR = np.matmul(b.T,np.linalg.inv(A))

print('time to run: ',time.time()-start_time)

time to run:  0.06400418281555176


In [9]:
page_rank = {keys[n]:PR[n] for n in range(len(PR))}
page_rank = {url: rank for url, rank in sorted(page_rank.items(), key=lambda item: item[1], reverse=True)}
[(key,page_rank[key]) for key in page_rank.keys()][:10]

[('/wiki/Main_Page', 4.523615406044074),
 ('/wiki/Black_hole', 1.8426238105143027),
 ('/wiki/Supernova', 1.7004531845537845),
 ('/wiki/Dark_matter', 1.684915610039688),
 ('/wiki/General_relativity', 1.6588876835802202),
 ('/wiki/Universe', 1.5643507207699832),
 ('/wiki/Galaxy', 1.5012556319362764),
 ('/wiki/Neutron_star', 1.4945359986560993),
 ('/wiki/White_dwarf', 1.4860015239025481),
 ('/wiki/Dark_energy', 1.3737415360304062)]

<h1>Random Walk</h1>

In [ ]:
steps = 10
walkers = 10**4
creeper_iterations = 2

start_time = time.time()

urls = list(url_targets.keys())

#initial positions
for walker in range(walkers):
    positions = np.random.choice(urls,walkers)
    
for step in range(steps):
    for walker in range(walkers):
        positions[walker] = np.random.choice(list(url_targets[positions[walker]]))
        
page_rank = {url: (1-d) + d*np.sum(positions == url)/walkers for url in urls}
print('time to run: ',time.time()-start_time)

In [ ]:
page_rank = {url: rank for url, rank in sorted(page_rank.items(), key=lambda item: item[1], reverse=True)}
[(key,page_rank[key]) for key in page_rank.keys()][:10]